WEB SCRAPPING FOTMOB

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:

html = '''
<!-- Aquí iría el HTML que contiene los datos -->
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>3</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Goles</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>4</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Asistencias</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Comenzado</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>5</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Partidos</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><span>443</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Minutos jugados</span></div>
<div class="css-1kkujsn-StatBox e1ahduwc6"><div class="css-1viv3lr-PlayerRatingStyled e1v8brex0"><span>8,66</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Puntuación</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_yellow_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#ffce2c" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>1</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas amarillas</span></div>
<div class="css-1bw13og-StatBox e1ahduwc6"><div class="css-170fd60-StatValue e1ahduwc5"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 20 20"><g id="prefix__ic_red_card" transform="translate(-700 -508)"><rect id="prefix__Rectangle_2316" width="11.111" height="15.556" fill="#e55e5b" data-name="Rectangle 2316" rx="2.5" transform="translate(704.444 510.222)"></rect><path id="prefix__Rectangle_2688" fill="none" d="M0 0H20V20H0z" data-name="Rectangle 2688" transform="translate(700 508)"></path></g></svg><span>0</span></div><span class="css-1m9s03m-StatTitle e1ahduwc4">Tarjetas rojas</span></div>
'''

# Cargar el HTML con BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Encontrar todas las estadísticas
stats = soup.find_all('div', class_='css-1kkujsn-StatBox')

# Iterar sobre cada estadística
for stat in stats:
    # Intentar extraer el valor
    value_div = stat.find('div', class_='css-170fd60-StatValue')
    if value_div:
        value = value_div.get_text(strip=True)
    else:
        # Para la puntuación
        value_div = stat.find('div', class_='css-1viv3lr-PlayerRatingStyled')
        value = value_div.get_text(strip=True) if value_div else 'N/A'

    # Extraer el tipo de estadística
    stat_type = stat.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
    print(f'{stat_type}: {value}')

# También extraer tarjetas que tienen una estructura diferente
cards = soup.find_all('div', class_='css-1bw13og-StatBox')
for card in cards:
    value_div = card.find('span')  # Obtener el valor del span directamente
    value = value_div.get_text(strip=True) if value_div else 'N/A'
    stat_type = card.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
    print(f'{stat_type}: {value}')


Goles: 3
Asistencias: 4
Comenzado: 5
Partidos: 5
Minutos jugados: 443
Puntuación: 8,66
Tarjetas amarillas: 1
Tarjetas rojas: 0


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def obtener_datos_jugador(url_jugador):
    # Definir headers para simular un navegador real
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/112.0.0.0 Safari/537.36'
    }

    try:
        # Realizar la solicitud HTTP a la página del jugador
        response = requests.get(url_jugador, headers=headers)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud HTTP: {e}")
        return None

    # Crear un objeto BeautifulSoup para analizar el HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer el nombre del jugador
    nombre_jugador_tag = soup.find('h1', class_='css-zt63wq-PlayerNameCSS e1i691m81')
    nombre_jugador = nombre_jugador_tag.get_text(strip=True) if nombre_jugador_tag else None

    # Extraer el nombre del equipo
    nombre_equipo_tag = soup.find('div', class_='css-14k6s2u-TeamCSS e1i691m82')
    nombre_equipo = nombre_equipo_tag.get_text(strip=True) if nombre_equipo_tag else None

    # Extraer estadísticas del jugador (altura, número, edad, etc.)
    stats = soup.find_all('div', class_='css-1u6v53x-PlayerBioStatCSS')
    datos_estadisticas = {}
    
    for stat in stats:
        value = stat.find('div', class_='css-to3w1c-StatValueCSS').get_text(strip=True)
        stat_type = stat.find('div', class_='css-10h4hmz-StatTitleCSS').get_text(strip=True)
        datos_estadisticas[stat_type] = value

    # Extraer estadísticas de rendimiento (goles, asistencias, etc.)
    stats_rendimiento = soup.find_all('div', class_='css-1kkujsn-StatBox')
    
    for stat in stats_rendimiento:
        value_div = stat.find('div', class_='css-170fd60-StatValue')
        if value_div:
            value = value_div.get_text(strip=True)
        else:
            # Para la puntuación
            value_div = stat.find('div', class_='css-1viv3lr-PlayerRatingStyled')
            value = value_div.get_text(strip=True) if value_div else 'N/A'

        stat_type = stat.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
        datos_estadisticas[stat_type] = value

    # También extraer tarjetas (amarillas, rojas)
    cards = soup.find_all('div', class_='css-1bw13og-StatBox')
    
    for card in cards:
        value_div = card.find('span')  # Obtener el valor del span directamente
        value = value_div.get_text(strip=True) if value_div else 'N/A'
        stat_type = card.find('span', class_='css-1m9s03m-StatTitle').get_text(strip=True)
        datos_estadisticas[stat_type] = value

    # Combinar todos los datos en un diccionario
    datos_jugador = {
        'nombre_jugador': nombre_jugador,
        'nombre_equipo': nombre_equipo
    }
    datos_jugador.update(datos_estadisticas)

    return datos_jugador


if __name__ == "__main__":
    # Lista de URLs de jugadores
    urls_jugadores = [
        "https://www.fotmob.com/es/players/1467236",
        # Puedes añadir más URLs de jugadores aquí
    ]
    
    # Lista para almacenar todos los datos de los jugadores
    lista_datos_jugadores = []

    # Iterar sobre las URLs de jugadores y obtener los datos
    for url in urls_jugadores:
        datos = obtener_datos_jugador(url)
        if datos:
            lista_datos_jugadores.append(datos)
    


In [18]:
df = pd.DataFrame(lista_datos_jugadores)
df


,nombre_jugador,nombre_equipo,Altura,Camiseta,13 jul 2007,Pie preferido,Goles,Asistencias,Comenzado,Partidos,Minutos jugados,Puntuación,Tarjetas amarillas,Tarjetas rojas
0,Lamine Yamal,Barcelona,180 cm,19,17 años,Izquierdo,3,5,7,7,623,"8,45",1,0
